In [1]:
import os
import sys
import json
import codecs

import time
from datetime import datetime

In [16]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark import SparkFiles
from pyspark.sql.functions import col, lit, length, row_number, when
from pyspark.sql.functions import upper, lower

In [3]:
from lib import spark_utils

In [4]:
spark = spark_utils.get_spark()

25/03/17 10:15:42 WARN Utils: Your hostname, Mac-MD2XX1D4WV.local resolves to a loopback address: 127.0.0.1; using 192.168.11.215 instead (on interface en0)
25/03/17 10:15:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 10:15:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [6]:
import os
import glob
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lit, length, row_number, when
from pyspark.sql.functions import avg, count, sum
from pyspark.sql.functions import upper, lower
from pyspark.sql.functions import substring, split
from pyspark.sql import functions
from pyspark.sql.types import IntegerType, StringType, LongType
from pyspark.sql.types import StructType, StructField

In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.width', 4000)

In [8]:
def cast_to_int_with_default(value):
    try:
        if value is None:
            return 0  # or any other default value you prefer
        else:
            return int(value)
    except ValueError:
        return 0

from pyspark.sql.functions import udf

udf_cast_to_int_with_default = udf(cast_to_int_with_default, IntegerType())

In [71]:
def get_joined_data_date(spark, input_path, _year, _qurt):
    coverpage_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(
        input_path % (_year, _qurt, 'COVERPAGE.tsv'))

    coverpage_ = coverpage_.withColumns({'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    infotable_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(
        input_path % (_year, _qurt, 'INFOTABLE.tsv'))

    # cusup 合法性检查
    infotable_ = infotable_.withColumn('CUSIP', lower(col('CUSIP')))
    infotable_ = infotable_.filter((infotable_['SSHPRNAMTTYPE'] == 'SH') &
                                   (infotable_['PUTCALL'].isNull()) &
                                   (infotable_['CUSIP'] != '000000000') &
                                   (infotable_['CUSIP'] != '0000000na'))
    infotable_ = infotable_.filter(length(infotable_['CUSIP']) == 9)

    # 仅选取普通股
    infotable_ = infotable_.filter(upper(col("TITLEOFCLASS")).contains("COM"))
    infotable_ = infotable_.withColumns({
        'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    summarypage_ = spark.read.option('delimiter', '\t').option(
        'header', True).csv(input_path % (_year, _qurt, 'SUMMARYPAGE.tsv'))

    summarypage_ = summarypage_.withColumns({
        'YEAR': lit(_year),
        'QUARTER': lit(_qurt)})

    submission_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(input_path % (_year, _qurt, 'SUBMISSION.tsv'))
    submission_ = submission_.withColumns({'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    joined_ = infotable_.join(
        coverpage_, (infotable_['ACCESSION_NUMBER'] == coverpage_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == coverpage_['YEAR']) & (infotable_['QUARTER'] == coverpage_['QUARTER']),
        'left'
    ).join(
        summarypage_, (infotable_['ACCESSION_NUMBER'] == summarypage_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == summarypage_['YEAR']) & (infotable_['QUARTER'] == summarypage_['QUARTER']),
        'left'
    ).join(
        submission_, (infotable_['ACCESSION_NUMBER'] == submission_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == submission_['YEAR']) & (infotable_['QUARTER'] == submission_['QUARTER']),
        'left'
    ).drop(
        coverpage_['YEAR'], coverpage_['QUARTER'], coverpage_['ACCESSION_NUMBER'],
        summarypage_['YEAR'], summarypage_['QUARTER'], summarypage_['ACCESSION_NUMBER'],
        submission_['YEAR'], submission_['QUARTER'], submission_['ACCESSION_NUMBER'])

    joined_ = joined_.withColumns({
        'NAMEOFISSUER': upper(col('NAMEOFISSUER')),
        'FILINGMANAGER_NAME': upper(col('FILINGMANAGER_NAME'))
    })

    # 需要有一个去重判断,投资机构可能多次上传,13HR,13HR-A,...
    window_spec = Window.partitionBy('CUSIP', 'FILINGMANAGER_NAME').orderBy(col('FILING_DATE').desc())
    joined_ = joined_.withColumn('row_number', row_number().over(window_spec))
    
    xxxx = joined_.select('*')
    
    joined_ = joined_.filter(col('row_number') == 1).drop('row_number')

    return joined_,xxxx

In [72]:
def transfer_standard_unit(filter_data, year):
    filter_data = filter_data.withColumns({
        'VALUE': udf_cast_to_int_with_default(col('VALUE')),
        'SSHPRNAMT': udf_cast_to_int_with_default(col('SSHPRNAMT')),
    })
    # 2022年及以前的VALUE单位为千$，2023年及之后为$(实际从1月3号开始)
    filter_data = filter_data.withColumn(
        'VALUE', when(lit(year) >= 2023, col('VALUE')).otherwise(col('VALUE')*1000))
    return filter_data

In [73]:
def filter_data_by_share_value(filter_data, year):
    # 对空值或负值填0
    filter_data = filter_data.withColumns({
        "VALUE": when(col("VALUE").isNull() | (col("VALUE") < 0), 0).otherwise(col("VALUE")),
        "SSHPRNAMT": when(col("SSHPRNAMT").isNull() | (col("SSHPRNAMT") < 0), 0).otherwise(col("SSHPRNAMT")),
    })
    # 以股票为Key统计总被交易价值、效果量
    ticker_value = filter_data.groupby(['CUSIP']).agg(
        sum(col('VALUE')).alias('VALUE'),
        sum(col('SSHPRNAMT')).alias('SSHPRNAMT')
    ).withColumn(
        'VPSSH', col('VALUE') / col('SSHPRNAMT')
    ).filter(
        (col('VPSSH') > 0.00000000000001) & (col('VPSSH') < 100000000000.)
    )

    ticker_value = ticker_value.withColumnRenamed(
        'CUSIP', 'CUSIP_ticker'
    ).withColumnRenamed(
        'VALUE', 'VALUE_ticker'
    ).withColumnRenamed(
        'SSHPRNAMT', 'SSHPRNAMT_ticker'
    )

    # 如果基金持有成本明显偏离总平均成本，认为是脏数据，过滤掉
    filter_data = filter_data.join(
        ticker_value, filter_data['CUSIP'] == ticker_value['CUSIP_ticker'], 'left'
    ).drop('CUSIP_ticker', 'VALUE_ticker', 'SSHPRNAMT_ticker')

    filter_data = filter_data.filter(
        (col('VALUE') / col('SSHPRNAMT') > 0.1 * col('VPSSH'))
        & (col('VALUE') / col('SSHPRNAMT') < 10 * col('VPSSH')))

    filter_data = filter_data.drop('VPSSH')

    return filter_data



In [74]:
input_path = '/Users/liuda/Library/CloudStorage/Dropbox/shareit/code/trading/data/hedge/%sq%s_form13f/%s'
cur_year='2024'
cur_qurt='2'
pre_year='2024'
pre_qurt='1'

In [75]:
# 用join curr的逻辑处理2024 q2

In [76]:
joined_cur, xxx = get_joined_data_date(spark, input_path, cur_year, cur_qurt)

In [77]:
xxx.groupby(['row_number']).agg(sum(lit(1))).show()

+----------+------+
|row_number|sum(1)|
+----------+------+
|       148|    42|
|       463|    17|
|       471|    17|
|       496|    17|
|       833|    10|
|      1088|    10|
|      1238|     1|
|      1342|     1|
|      1580|     1|
|      1591|     1|
|       243|    24|
|       392|    19|
|       540|    16|
|       623|    13|
|       737|    12|
|       858|    10|
|       897|    10|
|      1025|    10|
|      1084|    10|
|      1127|    10|
+----------+------+
only showing top 20 rows



In [81]:
# xxx.filter(col('row_number') > 10).limit(100).toPandas()
# xxx.filter((col('CUSIP') == '00081t108') & (col('FILINGMANAGER_NAME') == 'METLIFE INVESTMENT MANAGEMENT, LLC')).limit(100).toPandas()
xxx.filter((col('CUSIP') == '001055102') & (col('FILINGMANAGER_NAME') == 'ORG PARTNERS LLC')).limit(100).toPandas()

,ACCESSION_NUMBER,INFOTABLE_SK,NAMEOFISSUER,TITLEOFCLASS,CUSIP,FIGI,VALUE,SSHPRNAMT,SSHPRNAMTTYPE,PUTCALL,INVESTMENTDISCRETION,OTHERMANAGER,VOTING_AUTH_SOLE,VOTING_AUTH_SHARED,VOTING_AUTH_NONE,YEAR,QUARTER,REPORTCALENDARORQUARTER,ISAMENDMENT,AMENDMENTNO,AMENDMENTTYPE,CONFDENIEDEXPIRED,DATEDENIEDEXPIRED,DATEREPORTED,REASONFORNONCONFIDENTIALITY,FILINGMANAGER_NAME,FILINGMANAGER_STREET1,FILINGMANAGER_STREET2,FILINGMANAGER_CITY,FILINGMANAGER_STATEORCOUNTRY,FILINGMANAGER_ZIPCODE,REPORTTYPE,FORM13FFILENUMBER,CRDNUMBER,SECFILENUMBER,PROVIDEINFOFORINSTRUCTION5,ADDITIONALINFORMATION,OTHERINCLUDEDMANAGERSCOUNT,TABLEENTRYTOTAL,TABLEVALUETOTAL,ISCONFIDENTIALOMITTED,FILING_DATE,SUBMISSIONTYPE,CIK,PERIODOFREPORT,row_number
0,0001979372-24-000002,99912826,AFLAC INC,COMMON STOCK,001055102,None,3407,40,SH,None,SOLE,None,3407,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,1
1,0001979372-24-000002,99912827,AFLAC INC,COMMON STOCK,001055102,None,3407,40,SH,None,SOLE,None,3407,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,2
2,0001979372-24-000002,99914153,AFLAC INC,COMMON STOCK,001055102,None,1618,19,SH,None,SOLE,None,1618,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,3
3,0001979372-24-000002,99914154,AFLAC INC,COMMON STOCK,001055102,None,5111,60,SH,None,SOLE,None,5111,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,4
4,0001979372-24-000002,99913171,AFLAC INC,COMMON STOCK,001055102,None,539,6,SH,None,SOLE,None,539,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,5
5,0001979372-24-000002,99914233,AFLAC INC,COMMON STOCK,001055102,None,937,11,SH,None,SOLE,None,937,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,6
6,0001979372-24-000002,99914234,AFLAC INC,COMMON STOCK,001055102,None,1788,21,SH,None,SOLE,None,1788,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,7
7,0001979372-24-000002,99914235,AFLAC INC,COMMON STOCK,001055102,None,4685,55,SH,None,SOLE,None,4685,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,8
8,0001979372-24-000002,99914236,AFLAC INC,COMMON STOCK,001055102,None,681,8,SH,None,SOLE,None,681,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,9
9,0001979372-24-000002,99913321,AFLAC INC,COMMON STOCK,001055102,None,2640,31,SH,None,SOLE,None,2640,0,0,2024,2,31-MAR-2024,None,None,None,None,None,None,None,ORG PARTNERS LLC,13548 ZUBRICK RD,None,ROANOKE,IN,46783,13F HOLDINGS REPORT,028-23285,None,None,N,None,0,69228,10217692126,N,06-MAY-2024,13F-HR,0001979372,31-MAR-2024,10


In [38]:
joined_cur = transfer_standard_unit(joined_cur, cur_year)

In [39]:
joined_cur = filter_data_by_share_value(joined_cur, cur_year)

In [40]:
joined_cur.filter((col('CUSIP') == '007903107') 
                  & (col('NAMEOFISSUER') == 'ADVANCED MICRO DEVICES INC')
                  & (col('FILINGMANAGER_NAME') == 'ZWJ INVESTMENT COUNSEL INC')).limit(3).toPandas()

,ACCESSION_NUMBER,INFOTABLE_SK,NAMEOFISSUER,TITLEOFCLASS,CUSIP,FIGI,VALUE,SSHPRNAMT,SSHPRNAMTTYPE,PUTCALL,INVESTMENTDISCRETION,OTHERMANAGER,VOTING_AUTH_SOLE,VOTING_AUTH_SHARED,VOTING_AUTH_NONE,YEAR,QUARTER,REPORTCALENDARORQUARTER,ISAMENDMENT,AMENDMENTNO,AMENDMENTTYPE,CONFDENIEDEXPIRED,DATEDENIEDEXPIRED,DATEREPORTED,REASONFORNONCONFIDENTIALITY,FILINGMANAGER_NAME,FILINGMANAGER_STREET1,FILINGMANAGER_STREET2,FILINGMANAGER_CITY,FILINGMANAGER_STATEORCOUNTRY,FILINGMANAGER_ZIPCODE,REPORTTYPE,FORM13FFILENUMBER,CRDNUMBER,SECFILENUMBER,PROVIDEINFOFORINSTRUCTION5,ADDITIONALINFORMATION,OTHERINCLUDEDMANAGERSCOUNT,TABLEENTRYTOTAL,TABLEVALUETOTAL,ISCONFIDENTIALOMITTED,FILING_DATE,SUBMISSIONTYPE,CIK,PERIODOFREPORT
0,0001580642-24-002320,99721565,ADVANCED MICRO DEVICES INC,COM,007903107,None,336794,1866,SH,None,SOLE,None,0,0,1866,2024,2,31-MAR-2024,N,None,None,None,None,None,None,ZWJ INVESTMENT COUNSEL INC,75 14th Street Ne,Suite 2900,Atlanta,GA,30309-7604,13F HOLDINGS REPORT,028-02376,None,None,N,None,0,227,2196266229,None,26-APR-2024,13F-HR,0000842782,31-MAR-2024


In [18]:
##############该部分为测试用 #################### 
# 用join pre的逻辑处理2024q2,能确认两个逻辑处理完的数据是一致的 ,
# joined_pre = get_joined_data_date(spark, input_path, # , cur_qurt)   # 过滤CUSIP
# joined_pre = joined_pre.select(['NAMEOFISSUER', 'FILINGMANAGER_NAME', 'CUSIP', 'YEAR', 'QUARTER', 'VALUE','SSHPRNAMTTYPE', 'SSHPRNAMT','FILING_DATE'])
# joined_pre = transfer_standard_unit(joined_pre, cur_year)
# joined_pre = filter_data_by_share_value(joined_pre, cur_year)
# 
# joined_pre = joined_pre.withColumnRenamed(
#     "VALUE", "PREVALUE").withColumnRenamed(
#     'YEAR', 'PREYEAR').withColumnRenamed(
#     'QUARTER', 'PREQUARTER').withColumnRenamed(
#     'VALUE', 'PREVALUE').withColumnRenamed(
#     'SSHPRNAMTTYPE', 'PRESSHPRNAMTTYPE').withColumnRenamed(
#     'SSHPRNAMT', 'PRESSHPRNAMT').withColumnRenamed(
#     'FILING_DATE', 'PRE_FILING_DATE')

# joined_pre.filter((col('CUSIP') == '007903107') 
#                   & (col('NAMEOFISSUER') == 'ADVANCED MICRO DEVICES INC')
#                   & (col('FILINGMANAGER_NAME') == 'ZWJ INVESTMENT COUNSEL INC')).limit(3).toPandas()

#############测试部分结束##################### 

In [19]:
# 正常处理pre数据 

In [20]:
joined_pre = get_joined_data_date(spark, input_path, pre_year, pre_qurt)   # 过滤CUSIP

joined_pre = joined_pre.select(['NAMEOFISSUER', 'FILINGMANAGER_NAME', 'CUSIP',
                                'YEAR', 'QUARTER', 'VALUE',
                                'SSHPRNAMTTYPE', 'SSHPRNAMT',
                                'FILING_DATE'])

# 统一VALUE单位
joined_pre = transfer_standard_unit(joined_pre, pre_year)

# 统计股票的均价，如果基金成本明显异于均价，认为是异常值,过滤
joined_pre = filter_data_by_share_value(joined_pre, pre_year)

joined_pre = joined_pre.withColumnRenamed(
    "VALUE", "PREVALUE").withColumnRenamed(
    'YEAR', 'PREYEAR').withColumnRenamed(
    'QUARTER', 'PREQUARTER').withColumnRenamed(
    'VALUE', 'PREVALUE').withColumnRenamed(
    'SSHPRNAMTTYPE', 'PRESSHPRNAMTTYPE').withColumnRenamed(
    'SSHPRNAMT', 'PRESSHPRNAMT').withColumnRenamed(
    'FILING_DATE', 'PRE_FILING_DATE')

In [21]:
# join当前qt和last qt 

In [22]:
joined_data = joined_cur.join(
    joined_pre, (joined_cur['FILINGMANAGER_NAME'] == joined_pre['FILINGMANAGER_NAME']) &
    (joined_cur['CUSIP'] == joined_pre['CUSIP']),
    'left'
).drop(
    joined_pre['FILINGMANAGER_NAME'], joined_pre['NAMEOFISSUER'], joined_pre['CUSIP']
).withColumn('ADDITIONALINFORMATION', substring('ADDITIONALINFORMATION', 0, 2000))

joined_data = joined_data.withColumns({
    "VALUE": when(col("VALUE").isNull() | (col("VALUE") < 0), 0).otherwise(col("VALUE")),
    "PREVALUE": when(col("PREVALUE").isNull() | (col("PREVALUE") < 0), 0).otherwise(col("PREVALUE")),
    "SSHPRNAMT": when(col("SSHPRNAMT").isNull() | (col("SSHPRNAMT") < 0), 0).otherwise(col("SSHPRNAMT")),
    "PRESSHPRNAMT": when(col("PRESSHPRNAMT").isNull() | (col("PRESSHPRNAMT") < 0), 0).otherwise(col("PRESSHPRNAMT"))
})

In [23]:
def get_manager_level(spark):
    # 获取基金的历史体量数据，依赖上一年的数据产出,所有跑数据要从小年份开始跑
    input_data_dirs = []
    for stat_year in '2020,2021,2022,2023'.split(','):
        input_data_dirs += glob.glob('{base_dir}/{year}q*_form13f/manager/'.format(
            base_dir='/Users/liuda/Local/data/trading/data/hedgeparsing', year=int(stat_year)))

    schema = StructType([
        StructField("FILINGMANAGER_NAME", StringType(), nullable=True),
        StructField("MANAGER_VALUE", IntegerType(), nullable=True),
        StructField("MANAGER_PREVALUE", IntegerType(), nullable=True),
        StructField("MANAGER_LEVEL", IntegerType(), nullable=True),
        StructField("YEAR", IntegerType(), nullable=True),
        StructField("QUARTER", IntegerType(), nullable=True)
    ])
    #
    data = spark.read.schema(schema).option('delimiter', '\t').option('header', True).csv(input_data_dirs)
    data = data.groupby(['FILINGMANAGER_NAME']).agg(avg(col('MANAGER_VALUE')).alias('MANAGER_VALUE'))

    quantiles = data.approxQuantile("MANAGER_VALUE", [0.99, 0.90, 0.70, 0.30, 0.0], 0.01)
    data = data.withColumn("MANAGER_LEVEL",
                           when(col("MANAGER_VALUE") > quantiles[0], 0)
                           .when(col("MANAGER_VALUE") > quantiles[1], 1)
                           .when(col("MANAGER_VALUE") > quantiles[2], 2)
                           .when(col("MANAGER_VALUE") > quantiles[3], 3)
                           .when(col("MANAGER_VALUE") >= quantiles[4], 4)
                           .otherwise(5))   # 5: 采用固定数据后有些ManagerLevel找不到，归入5

    return data

In [24]:
manager_history = get_manager_level(spark)

In [25]:
joined_data = joined_data.join(
    manager_history,
    (joined_data['FILINGMANAGER_NAME'] == manager_history['FILINGMANAGER_NAME']),
    'left'
).drop(manager_history['FILINGMANAGER_NAME'])

In [26]:
joined_data.persist()

DataFrame[ACCESSION_NUMBER: string, INFOTABLE_SK: string, NAMEOFISSUER: string, TITLEOFCLASS: string, CUSIP: string, FIGI: string, VALUE: int, SSHPRNAMT: int, SSHPRNAMTTYPE: string, PUTCALL: string, INVESTMENTDISCRETION: string, OTHERMANAGER: string, VOTING_AUTH_SOLE: string, VOTING_AUTH_SHARED: string, VOTING_AUTH_NONE: string, YEAR: string, QUARTER: string, REPORTCALENDARORQUARTER: string, ISAMENDMENT: string, AMENDMENTNO: string, AMENDMENTTYPE: string, CONFDENIEDEXPIRED: string, DATEDENIEDEXPIRED: string, DATEREPORTED: string, REASONFORNONCONFIDENTIALITY: string, FILINGMANAGER_NAME: string, FILINGMANAGER_STREET1: string, FILINGMANAGER_STREET2: string, FILINGMANAGER_CITY: string, FILINGMANAGER_STATEORCOUNTRY: string, FILINGMANAGER_ZIPCODE: string, REPORTTYPE: string, FORM13FFILENUMBER: string, CRDNUMBER: string, SECFILENUMBER: string, PROVIDEINFOFORINSTRUCTION5: string, ADDITIONALINFORMATION: string, OTHERINCLUDEDMANAGERSCOUNT: string, TABLEENTRYTOTAL: string, TABLEVALUETOTAL: string

In [27]:
joined_data.filter(col('CUSIP')=='007903107').limit(1).toPandas()

,ACCESSION_NUMBER,INFOTABLE_SK,NAMEOFISSUER,TITLEOFCLASS,CUSIP,FIGI,VALUE,SSHPRNAMT,SSHPRNAMTTYPE,PUTCALL,INVESTMENTDISCRETION,OTHERMANAGER,VOTING_AUTH_SOLE,VOTING_AUTH_SHARED,VOTING_AUTH_NONE,YEAR,QUARTER,REPORTCALENDARORQUARTER,ISAMENDMENT,AMENDMENTNO,AMENDMENTTYPE,CONFDENIEDEXPIRED,DATEDENIEDEXPIRED,DATEREPORTED,REASONFORNONCONFIDENTIALITY,FILINGMANAGER_NAME,FILINGMANAGER_STREET1,FILINGMANAGER_STREET2,FILINGMANAGER_CITY,FILINGMANAGER_STATEORCOUNTRY,FILINGMANAGER_ZIPCODE,REPORTTYPE,FORM13FFILENUMBER,CRDNUMBER,SECFILENUMBER,PROVIDEINFOFORINSTRUCTION5,ADDITIONALINFORMATION,OTHERINCLUDEDMANAGERSCOUNT,TABLEENTRYTOTAL,TABLEVALUETOTAL,ISCONFIDENTIALOMITTED,FILING_DATE,SUBMISSIONTYPE,CIK,PERIODOFREPORT,PREYEAR,PREQUARTER,PREVALUE,PRESSHPRNAMTTYPE,PRESSHPRNAMT,PRE_FILING_DATE,MANAGER_VALUE,MANAGER_LEVEL
0,0001767898-24-000002,99375386,ADVANCED MICRO DEVICES INC,COM,007903107,None,296004,1640,SH,None,SOLE,None,0,0,1640,2024,2,31-MAR-2024,N,None,None,None,None,None,None,"AMERICAN FINANCIAL ADVISORS, LLC",2551 Roswell Road,Suite 310,Marietta,GA,30062,13F HOLDINGS REPORT,028-19419,000115824,801-60761,N,None,0,151,709821088,None,22-APR-2024,13F-HR,0001767898,31-MAR-2024,2024,1,235119,SH,1595,31-JAN-2024,5.511180e+07,4


In [28]:
sel_df = joined_data.filter(col('CUSIP') == '007903107').groupby(
    ['YEAR', 'QUARTER', 'CUSIP', 'MANAGER_LEVEL']
).agg(
    sum('VALUE'),
    sum('PREVALUE')
)

In [29]:
sel_df.limit(100).toPandas()

,YEAR,QUARTER,CUSIP,MANAGER_LEVEL,sum(VALUE),sum(PREVALUE)
0,2024,2,007903107,4.0,2023528157,969479648
1,2024,2,007903107,NaN,1156939692,624273549
2,2024,2,007903107,3.0,1701960891,1300316551
3,2024,2,007903107,1.0,6871894006,5629449489
4,2024,2,007903107,2.0,3719251780,2944971413
5,2024,2,007903107,5.0,38197944955,29632509399


In [74]:
def get_joined_data_date(spark, input_path, _year, _qurt):
    coverpage_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(
        input_path % (_year, _qurt, 'COVERPAGE.tsv'))

    coverpage_ = coverpage_.withColumns({'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    infotable_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(
        input_path % (_year, _qurt, 'INFOTABLE.tsv'))

    # cusup 合法性检查
    infotable_ = infotable_.withColumn('CUSIP', lower(col('CUSIP')))
    infotable_ = infotable_.filter((infotable_['SSHPRNAMTTYPE'] == 'SH') &
                                   (infotable_['PUTCALL'].isNull()) &
                                   (infotable_['CUSIP'] != '000000000') &
                                   (infotable_['CUSIP'] != '0000000na'))
    infotable_ = infotable_.filter(length(infotable_['CUSIP']) == 9)

    # 仅选取普通股
    infotable_ = infotable_.filter(upper(col("TITLEOFCLASS")).contains("COM"))
    infotable_ = infotable_.withColumns({
        'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    summarypage_ = spark.read.option('delimiter', '\t').option(
        'header', True).csv(input_path % (_year, _qurt, 'SUMMARYPAGE.tsv'))

    summarypage_ = summarypage_.withColumns({
        'YEAR': lit(_year),
        'QUARTER': lit(_qurt)})

    submission_ = spark.read.option(
        'delimiter', '\t').option(
        'header', True).csv(input_path % (_year, _qurt, 'SUBMISSION.tsv'))
    submission_ = submission_.withColumns({'YEAR': lit(_year), 'QUARTER': lit(_qurt)})

    joined_ = infotable_.join(
        coverpage_, (infotable_['ACCESSION_NUMBER'] == coverpage_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == coverpage_['YEAR']) & (infotable_['QUARTER'] == coverpage_['QUARTER']),
        'left'
    ).join(
        summarypage_, (infotable_['ACCESSION_NUMBER'] == summarypage_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == summarypage_['YEAR']) & (infotable_['QUARTER'] == summarypage_['QUARTER']),
        'left'
    ).join(
        submission_, (infotable_['ACCESSION_NUMBER'] == submission_['ACCESSION_NUMBER']) &
        (infotable_['YEAR'] == submission_['YEAR']) & (infotable_['QUARTER'] == submission_['QUARTER']),
        'left'
    ).drop(
        coverpage_['YEAR'], coverpage_['QUARTER'], coverpage_['ACCESSION_NUMBER'],
        summarypage_['YEAR'], summarypage_['QUARTER'], summarypage_['ACCESSION_NUMBER'],
        submission_['YEAR'], submission_['QUARTER'], submission_['ACCESSION_NUMBER'])

    joined_ = joined_.withColumns({
        'NAMEOFISSUER': upper(col('NAMEOFISSUER')),
        'FILINGMANAGER_NAME': upper(col('FILINGMANAGER_NAME')),
        'VALUE': col('VALUE').cast('integer')
    })

    # 需要有一个去重判断,投资机构可能多次上传,13HR,13HR-A,...
    window_spec = Window.partitionBy('CUSIP', 'FILINGMANAGER_NAME').orderBy(col('FILING_DATE').desc(), col('VALUE').desc())
    joined_ = joined_.withColumn('row_number', row_number().over(window_spec))
    # joined_ = joined_.filter(col('row_number') == 1).drop('row_number')

    return joined_

In [75]:
input_path = '/Users/liuda/Local/data/trading/data/hedge/' + '%sq%s_form13f/%s'

In [76]:
_year=2022
_qurt = 2

In [77]:
df = get_joined_data_date(spark, input_path, _year, _qurt)

In [78]:
df.filter(col('row_number') > 3).limit(10).toPandas()

,ACCESSION_NUMBER,INFOTABLE_SK,NAMEOFISSUER,TITLEOFCLASS,CUSIP,FIGI,VALUE,SSHPRNAMT,SSHPRNAMTTYPE,PUTCALL,...,ADDITIONALINFORMATION,OTHERINCLUDEDMANAGERSCOUNT,TABLEENTRYTOTAL,TABLEVALUETOTAL,ISCONFIDENTIALOMITTED,FILING_DATE,SUBMISSIONTYPE,CIK,PERIODOFREPORT,row_number
0,0000950123-22-004765,67884432,AAON INC,COM PAR $0.004,000360206,None,62,1105,SH,None,...,None,6,8554,22640456,N,09-MAY-2022,13F-HR,0000036270,31-MAR-2022,4
1,0000950123-22-004765,67884430,AAON INC,COM PAR $0.004,000360206,None,28,503,SH,None,...,None,6,8554,22640456,N,09-MAY-2022,13F-HR,0000036270,31-MAR-2022,5
2,0000950123-22-004765,67884434,AAON INC,COM PAR $0.004,000360206,None,1,14,SH,None,...,None,6,8554,22640456,N,09-MAY-2022,13F-HR,0000036270,31-MAR-2022,6
3,0000950123-22-005312,68448860,AAON INC,COM PAR $0.004,000360206,None,2,44,SH,None,...,None,3,4275,96686762,N,12-MAY-2022,13F-HR,0000713676,31-MAR-2022,4
4,0001104659-22-059997,69149522,AAON INC,COM PAR $0.004,000360206,None,396,7103,SH,None,...,None,5,16499,4227459554,None,13-MAY-2022,13F-HR,0000102909,31-MAR-2022,4
5,0001104659-22-059997,69149526,AAR CORP,COM,000361105,None,574,11843,SH,None,...,None,5,16499,4227459554,None,13-MAY-2022,13F-HR,0000102909,31-MAR-2022,4
6,0001104659-22-059997,69149527,AAR CORP,COM,000361105,None,72,1496,SH,None,...,None,5,16499,4227459554,None,13-MAY-2022,13F-HR,0000102909,31-MAR-2022,5
7,0000072971-22-000135,68251790,AAR CORP,COM,000361105,None,41,848,SH,None,...,None,7,19952,389911065,N,16-MAY-2022,13F-HR,0000072971,31-MAR-2022,4
8,0000950123-22-004585,67776465,ACCO BRANDS CORPORATION,Common Stock,00081t108,None,6,871,SH,None,...,None,46,12203,57496695,N,05-MAY-2022,13F-HR,0001692234,31-MAR-2022,4
9,0000895421-22-000458,68934651,ADMA BIOLOGICS INC,COM,000899104,None,17,9527,SH,None,...,None,18,32358,741466945,N,13-MAY-2022,13F-HR,0000895421,31-MAR-2022,4


In [79]:
window_spec = Window.partitionBy('CUSIP', 'FILINGMANAGER_NAME').orderBy(col('FILING_DATE').desc(), col('VALUE').desc())
df = df.withColumn('row_number', row_number().over(window_spec))

In [80]:
# df.printSchema()

In [81]:
sum_value = df.filter(F.col("VALUE").cast("integer").isNotNull()) \
              .select(F.sum(F.col("VALUE").cast("integer")).alias("total_sum"))

In [82]:
sum_value.show()

+------------+
|   total_sum|
+------------+
|101616998392|
+------------+

